In [1]:
from Bio.PDB import *
from Bio.PDB.DSSP import DSSP
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from Bio import SeqIO
from Bio.SeqUtils import seq1
import Bio

from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.SubsMat import MatrixInfo as matlist



import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy.linalg as linalg
from scipy import sparse
import scipy.signal
from snf import *

import pandas as pd

from difflib import SequenceMatcher


/Users/boyanxu/opt/anaconda3/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [2]:
df = pd.read_excel('FileS1.xlsx', skiprows = 9)


In [261]:
#gets pdb structures from pdb database
if False:
    pdbl = PDBList()
    for code in set(df['PDB ID + chain']):
        pdbl.retrieve_pdb_file(code[:4], file_format = 'pdb', pdir='pdb/LRRPredictor')

In [4]:
import warnings
warnings.filterwarnings("ignore")

# edge cases: '2Z66A', 3JB9J, 4CP6A, 2ELLA, '2CA6A'

LRR_d = {}

for ii in range(df.shape[0]):
    if ii%2:
        code = df.iloc[ii]['PDB ID + chain']
        if code != '2CA6A':
            pass
        pdb_code = code[:4]
        chain_id = code[4]
        entry = df.iloc[ii-1]['entry']
        motifs = df.iloc[ii]['entry']        
        print(code)

        prefixSize = 5
        matchingRecords = []
        for record in SeqIO.parse("pdb/LRRPredictor/pdb%s.ent"%pdb_code, "pdb-seqres"):                 
            if record.annotations["chain"].upper()==chain_id:
                delta = 0
                offset = -1
                while delta < 10 and offset < 0:
                    offset = record.seq[delta:].find(entry[delta:prefixSize+delta])
                    delta += 1
                if offset >=0:
                    matchingRecords.append((record, offset))
        if not matchingRecords:
            raise Exception ('did not find chain with matching prefix')
        elif len(matchingRecords)>1:
            raise Exception ('found more than one matching record')
        offset = matchingRecords[0][1]
        chain_id_cased = matchingRecords[0][0].annotations["chain"]       
                
        #get pdb sequence
        parser = PDBParser()
        structure = parser.get_structure(pdb_code, "pdb/LRRPredictor/pdb%s.ent"%pdb_code)        
        chains = []
        for model in structure:
            for chain in model:
                if chain.id == chain_id_cased:
                    chains.append(chain)
                    
        if len(chains) == 0:
            raise Exception ('no chains found')
        elif len(chains) == 1:
            chain = chains[0]
            pdbseq = ''.join([seq1(res.get_resname()) for res in chain.get_residues()])
        else:
            print('found too many chains:', len(chains))
            chain = chains[0]
            pdbseq = ''.join([seq1(res.get_resname()) for res in chain.get_residues()])            
        
        #try to align motif window to pdb sequence

        matrix = matlist.blosum62
        queryRadius = 5
        motifLocs = []
        for jj, char in enumerate(motifs):
            if char != '-':
                if jj-queryRadius<0:
                    lb = 0
                    ub = 2*queryRadius
                elif jj+queryRadius>len(entry):
                    lb = len(entry)-2*queryRadius
                    ub = len(entry)
                else:
                    lb = jj - queryRadius
                    ub = jj + queryRadius
                query = entry[lb:ub]
                
                motifLoc = pdbseq.find(query)+jj-lb
                
                if motifLoc < 0:
                    print('needed to do alignment')
                    print(query)
                    try:
                        a=pairwise2.align.localds(query, pdbseq, matrix, -1, -1, penalize_end_gaps=(False, False), one_alignment_only=True)[0]
                        print(format_alignment(*a))
                        # a = next(pairwise2.align.localdx(query, pdbseq, matrix, one_alignment_only=True))                        
                        # a = next(pairwise2.align.localds(query, pdbseq, matrix, -1, -1, penalize_end_gaps=(False, False), one_alignment_only=True))                        
                    except:
                        print(pdbseq)
                        raise Exception('alignment failed')
                    if a.score <queryRadius*2:
                        raise Exception('no alignments found')
                    
                    queryShift = int(format_alignment(*a).split(' ')[0])-1
                    start = int(format_alignment(*a).split(' ')[3].split('\n')[1])-1
                    motifLoc = start+jj-lb-queryShift
                    print(motifLoc)
                motifLocs.append(motifLoc)

        print(motifLocs)
        LRR_d[code] = motifLocs
    

1A9NA
[19, 42, 64, 88, 113, 140]
1H6UA
[42, 64, 86, 108, 130, 152, 174, 196, 218]
1IO0A
[18, 47, 75, 103, 133]
1JL5A
[39, 59, 79, 99, 121, 141, 163, 183, 205, 225, 245]
1O6VA
[45, 67, 89, 111, 133, 155, 176, 198, 220, 242]
1OGQA
[51, 77, 102, 126, 150, 175, 198, 222, 245]
1OZNA
[12, 33, 57, 81, 106, 130, 154, 178, 202, 226, 254]
1P9AG
[11, 32, 56, 78, 101, 125, 149, 173, 196]
1PGVA
[16, 45, 73, 101, 132]
1W8AA
[9, 30, 55, 79, 103, 127]
1WWLA
[5, 28, 65, 92, 118, 146, 170, 198, 225, 250]
1XEUA
[42, 64, 86, 107, 129, 151, 173]
1XKUA
[11, 32, 56, 80, 101, 125, 151, 172, 196, 220, 243]
1ZIWA
[2, 23, 47, 71, 95, 119, 143, 169, 193, 220, 246]
2ASTB
[42, 65, 88, 113, 137, 163, 190, 217, 242]
2BNHA
[25, 53, 82, 110, 139, 167, 196, 224, 253]
2CA6A
needed to do alignment
MARFSIEGKS
2 ARFSIEGKS
  |||||||||
1 ARFSIEGKS
  Score=43

-1
[-1, 32, 60, 94, 122, 159, 187, 216, 244, 274, 303, 332]
2ELLA
found too many chains: 20
[25, 50, 72, 96, 121, 148]
2FT3A
[10, 31, 55, 79, 100, 124, 149, 170, 194, 21

In [5]:
import pickle

with open('pickles/LRRPred_validation.pickle', 'wb') as handle:
    pickle.dump(LRR_d, handle, protocol = pickle.HIGHEST_PROTOCOL)


In [230]:
del LRR_d